In [ ]:
!pip install pandas

In [ ]:
!pip install mutagen

In [ ]:
!pip install bs4

In [70]:
import os
import pandas as pd
import librosa
from mutagen.mp3 import MP3
import zipfile
import requests
from bs4 import BeautifulSoup
import numpy as np

# data acquisition methods

Used a mixture of sourcing from current library for things like hessle audio and subscribed for 7 GBP to uknown to the unknown's bandcamp for full access to it's discography and used `bandcamp-dl` to batch download.
Used yt-dlp to download from clone.nl's youtube page and musicbrainz picard but at the moment finding auto-tagging is not particularly quick or accurate so currently tabling the analysis of clone in the scope of what I am turning in at the moment.

After messing around and looking into scraping between spotify, distributor sites, discogs, and last.fm for the given dataset and general trend that unless someone tags it in last.fm or it makes it to musicbrainz, per release genre via discogs is the best trade off between per-release precision and popuplating the field. granted many tagged genres will be listed. Sometimes more genres than there are tracks given the overlapping nature of aesthetic descriptions.

In [112]:
%load_ext autoreload
%autoreload 2

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

In [12]:
# this was to unzip the files the way `bandcamp-download` downloaded them
## installable via git clone and pip install . or poetry
## requires a logged in bandcamp browser window

def unzip_files_in_directory(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.zip'):
                zip_path = os.path.join(root, file)
                print(f"Unzipping: {zip_path}")
                with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                    # Extract all the contents into the directory containing the zip file
                    zip_ref.extractall(root)

directory_path = '/mnt/d/UTTU/'
unzip_files_in_directory(directory_path)


Unzipping: /mnt/d/UTTU/3STRANGE/3STRANGE - The New World.zip
Unzipping: /mnt/d/UTTU/3STRANGE, Alan Fitzpatrick/3STRANGE, Alan Fitzpatrick - Cyclone Jazz EP.zip
Unzipping: /mnt/d/UTTU/5kinAndBone5/5kinAndBone5 - Make U Understand.zip
Unzipping: /mnt/d/UTTU/9th House/9th House - 9th House - Phoenica + Loods Remix.zip
Unzipping: /mnt/d/UTTU/9th House/9th House - Planetary EP (Feat. Baltra & Fear-E).zip
Unzipping: /mnt/d/UTTU/Aaron Rutherford/Aaron Rutherford - Changing Contrast.zip
Unzipping: /mnt/d/UTTU/ABSOLUTE/ABSOLUTE. - Unarmed and Extremely Dangerous EP.zip
Unzipping: /mnt/d/UTTU/Addison Groove & Bim Sanga Present Bags Inc/Addison Groove & Bim Sanga Present Bags Inc. - Dance Trax Vol.17.zip
Unzipping: /mnt/d/UTTU/Alan Fitzpatrick/Alan Fitzpatrick - System Addict EP.zip
Unzipping: /mnt/d/UTTU/Alan Fitzpatrick/Alan Fitzpatrick - The Hole EP.zip
Unzipping: /mnt/d/UTTU/Alex Jann/Alex Jann - Cybernetik Memory.zip
Unzipping: /mnt/d/UTTU/Alex Jones/Alex Jones - Them Things.zip
Unzipping: /

In [13]:
# separate function to delete the archives after unzipping to make sure the first function worked fine before
## deleting the downloaded zips
def delete_zip_files(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.zip'):
                zip_path = os.path.join(root, file)
                print(f"Deleting ZIP file: {zip_path}")
                os.remove(zip_path)

delete_zip_files(directory_path)


Deleting ZIP file: /mnt/d/UTTU/3STRANGE/3STRANGE - The New World.zip
Deleting ZIP file: /mnt/d/UTTU/3STRANGE, Alan Fitzpatrick/3STRANGE, Alan Fitzpatrick - Cyclone Jazz EP.zip
Deleting ZIP file: /mnt/d/UTTU/5kinAndBone5/5kinAndBone5 - Make U Understand.zip
Deleting ZIP file: /mnt/d/UTTU/9th House/9th House - 9th House - Phoenica + Loods Remix.zip
Deleting ZIP file: /mnt/d/UTTU/9th House/9th House - Planetary EP (Feat. Baltra & Fear-E).zip
Deleting ZIP file: /mnt/d/UTTU/Aaron Rutherford/Aaron Rutherford - Changing Contrast.zip
Deleting ZIP file: /mnt/d/UTTU/ABSOLUTE/ABSOLUTE. - Unarmed and Extremely Dangerous EP.zip
Deleting ZIP file: /mnt/d/UTTU/Addison Groove & Bim Sanga Present Bags Inc/Addison Groove & Bim Sanga Present Bags Inc. - Dance Trax Vol.17.zip
Deleting ZIP file: /mnt/d/UTTU/Alan Fitzpatrick/Alan Fitzpatrick - System Addict EP.zip
Deleting ZIP file: /mnt/d/UTTU/Alan Fitzpatrick/Alan Fitzpatrick - The Hole EP.zip
Deleting ZIP file: /mnt/d/UTTU/Alex Jann/Alex Jann - Cyberneti

In [18]:
def find_album_and_song_genres_lastfm(api_key, artist, album):
    """
    This function uses the Last.fm API to find the genres of an album and its songs.

    Parameters:
    api_key (str): Your Last.fm API key.
    artist (str): The name of the artist.
    album (str): The title of the album.

    Returns:
    dict: A dictionary with album genres and song titles as keys and lists of genres as values.
    """
    url = "http://ws.audioscrobbler.com/2.0/"
    album_info_params = {
        "method": "album.getInfo",
        "api_key": api_key,
        "artist": artist,
        "album": album,
        "format": "json"
    }

    try:
        response = requests.get(url, params=album_info_params)
        response.raise_for_status()
        album_data = response.json()

        if 'album' not in album_data:
            return "Album not found or no data available."

        # Extracting album genres
        album_tags = album_data['album'].get('tags', {}).get('tag', [])
        album_genres = [tag['name'] for tag in album_tags]

        tracks = album_data['album'].get('tracks', {}).get('track', [])
        if not tracks:
            return {"Album Genres": album_genres, "Tracks": "No tracks found for the album."}

        # Initialize the result dictionary with album genres
        result = {"Album Genres": album_genres}

        # Adding genres for each track
        for track in tracks:
            track_name = track['name']
            track_info_params = {
                "method": "track.getInfo",
                "api_key": api_key,
                "artist": artist,
                "track": track_name,
                "format": "json"
            }

            track_response = requests.get(url, params=track_info_params)
            track_response.raise_for_status()
            track_data = track_response.json()

            if 'track' in track_data and 'toptags' in track_data['track']:
                tags = track_data['track']['toptags'].get('tag', [])
                genres = [tag['name'] for tag in tags]
                result[track_name] = genres
            else:
                result[track_name] = ["No genre tags available"]

        return result

    except requests.RequestException as e:
        return f"An error occurred: {e}"


In [14]:
def extract_features(file_path):
    # Load the audio file
    y, sr = librosa.load(file_path)

    # Extract features you are interested in
    tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
    timbre = librosa.feature.mfcc(y=y, sr=sr)

    # Other features...

    return {
        "tempo": tempo,
        "timbre": timbre.mean(axis=1)
    }


In [35]:
def read_tags(file_path):
    audio = MP3(file_path)
    tags = {
        'artist': None,
        'album': None,
        'year': None,
        'genre': None,
        'label': None,
        'title': None
    }

    # Directly iterate over tags
    # works for label *if* label is tagged in the metadata under copyright rather than `TPUB`
    for tag, value in audio.items():
        if tag.startswith('TPE1'):
            tags['artist'] = str(value)
        elif tag.startswith('TALB'):
            tags['album'] = str(value)
        elif tag.startswith('TDRC'):
            tags['year'] = str(value)
        elif tag.startswith('TCON'):
            tags['genre'] = str(value)
        elif tag.startswith('TCOP'):
            tags['label'] = str(value)
        elif tag.startswith('TIT2'):
            tags['title'] = str(value)

    return tags


In [36]:
testtag = read_tags("/mnt/d/MIR_music/Hessle/[HES005] Ramadanman - Blimey & Drowning & Tak (2008)/01. Ramadanman - Blimey.mp3")

In [24]:
testpath = "/mnt/d/MIR_music/Hessle/[HES005] Ramadanman - Blimey & Drowning & Tak (2008)/01. Ramadanman - Blimey.mp3"

In [37]:
testtag

{'artist': 'Ramadanman',
 'album': 'Blimey / Drowning (Version) / Tak',
 'year': '2008',
 'genre': 'Bass',
 'label': 'Hessle Audio',
 'title': 'Blimey'}

In [31]:
for i in audio:
    print(f"{i}\t {audio[i]}")

TALB	 Blimey / Drowning (Version) / Tak
TCOM	 David Kennedy
TCON	 Bass
TCOP	 Hessle Audio
TIT2	 Blimey
TPE1	 Ramadanman
TRCK	 01/03
TLAN	 Czech
TDRC	 2008


In [23]:
testtag

('Ramadanman', 'Blimey / Drowning (Version) / Tak', '2008', 'Bass', None)

In [97]:
import os

def process_files(folder_path, default_label=None):
    data = []

    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.mp3'):
                # Full path to the file
                full_file_path = os.path.join(root, file)

                tags = read_tags(full_file_path)

                # Use the label from the tags if available, otherwise use the default label
                label = tags['label'] if tags['label'] else default_label

                data.append({
                    "title": tags['title'],
                    "artist": tags['artist'],
                    "album": tags['album'],
                    "year": tags['year'],
                    "genre": tags['genre'],
                    "label": label,
                    "file_path": full_file_path
                })

    return pd.DataFrame(data)


In [98]:
UTTU_dir = "/mnt/d/UTTU/"

df_uttu = process_files(UTTU_dir, default_label="Unknown to the Unknown")

df_uttu.head()

,title,artist,album,year,genre,label,file_path
0,Modul8 (ItaloJohnson Remix),DJ Haus,None,2019,None,Unknown to the Unknown,/mnt/d/UTTU/DJ Haus - Modul8 (ItaloJohnson Rem...
1,Shadow Man,Neil Landstrumm,None,2019,None,Unknown to the Unknown,/mnt/d/UTTU/Neil Landstrumm - Shadow Man.mp3
2,Fluid Funk,Elisa Bee,Fluid Funk,2023,None,Unknown to the Unknown,/mnt/d/UTTU/'/Elisa Bee - Fluid Funk - 01 Flui...
3,Yes Yes,Elisa Bee,Fluid Funk,2023,None,Unknown to the Unknown,/mnt/d/UTTU/'/Elisa Bee - Fluid Funk - 02 Yes ...
4,Pins and Needles,Elisa Bee,Fluid Funk,2023,None,Unknown to the Unknown,/mnt/d/UTTU/'/Elisa Bee - Fluid Funk - 03 Pins...


In [107]:
hessle_dir = "/mnt/d/MIR_music/Hessle/"

df_hessle = process_files(hessle_dir, "Hessle Audio")

df_hessle.head()

0    /mnt/d/MIR_music/Hessle/116 & Rising (HESCD001...
1    /mnt/d/MIR_music/Hessle/116 & Rising (HESCD001...
2    /mnt/d/MIR_music/Hessle/116 & Rising (HESCD001...
3    /mnt/d/MIR_music/Hessle/116 & Rising (HESCD001...
4    /mnt/d/MIR_music/Hessle/116 & Rising (HESCD001...
Name: file_path, dtype: object

In [108]:
clone_dir = "/mnt/d/MIR_music/clone_ytdlp/"
df_clone = process_files(clone_dir, default_label="clone.nl")
df_clone.head()

,title,artist,album,year,genre,label,file_path
0,None,None,None,None,None,clone.nl,/mnt/d/MIR_music/clone_ytdlp/Cadans - Clocked ...
1,No Connection (Broken Mix),Cadans,No Connection,2019-04-05,None,clone.nl,/mnt/d/MIR_music/clone_ytdlp/Cadans - No Conne...
2,None,None,None,None,None,clone.nl,/mnt/d/MIR_music/clone_ytdlp/Cadans - No Conne...
3,None,None,None,None,None,clone.nl,/mnt/d/MIR_music/clone_ytdlp/Cadans - Uncuttab...
4,None,None,None,None,None,clone.nl,/mnt/d/MIR_music/clone_ytdlp/Clarence G - Hype...


In [116]:
import warnings

# Ignore specific warnings when g etting audio features, half works
warnings.filterwarnings('ignore', category=UserWarning, module='ipykernel')
warnings.filterwarnings('ignore', category=FutureWarning, module='librosa.core.audio')
warnings.filterwarnings('ignore', message='PySoundFile failed. Trying audioread instead.', category=UserWarning, module='librosa')

In [117]:
# add_audio_features(df_clone)
add_audio_features(df_hessle, hessle_dir)
add_audio_features(df_uttu, UTTU_dir)
add_audio_features(df_clone, clone_dir)

/tmp/ipykernel_5365/1776963190.py:3: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path)
/tmp/ipykernel_5365/1776963190.py:3: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path)
/tmp/ipykernel_5365/1776963190.py:3: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path)
/tmp/ipykernel_5365/1776963190.py:3: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path)
/tmp/ipykernel_5365/1776963190.py:3: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path)
/tmp/ipykernel_5365/1776963190.py:3: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path)
/tmp/ipykernel_5365/1776963190.py:3: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path)
/tmp/ipykernel_5365/1776963190.py:3: UserWarning: PySoundFile failed. Trying audioread ins

In [122]:
os.chdir("solo_final_project/")

In [123]:
df_clone.to_csv("clonedf.csv", index=False)
df_uttu.to_csv("uttudf.csv", index=False)
df_hessle.to_csv("hessle_audio.csv", index=False)

In [104]:
subset_df = df_uttu.head()

In [57]:
def fill_missing_genres(df, api_key):
    for index, row in df.iterrows():
        if not row['genre']:  # Check if genre is missing
            artist = row['artist']
            album = row['album']

            # Fetch genres from Last.fm
            genres_info = find_album_and_song_genres_lastfm(api_key, artist, album)

            # Update DataFrame
            if isinstance(genres_info, dict) and 'Album Genres' in genres_info:
                album_genres = genres_info['Album Genres']
                if album_genres:  # If genres are found
                    df.at[index, 'genre'] = ', '.join(album_genres)
                else:
                    df.at[index, 'genre'] = 'Unknown Genre'
            else:
                df.at[index, 'genre'] = 'Error or No Data'


In [61]:
import requests

def find_album_and_song_genres_lastfm(api_key, artist, album):
    url = "http://ws.audioscrobbler.com/2.0/"
    album_info_params = {
        "method": "album.getInfo",
        "api_key": api_key,
        "artist": artist,
        "album": album,
        "format": "json"
    }

    try:
        response = requests.get(url, params=album_info_params)
        response.raise_for_status()
        album_data = response.json()

        if 'album' not in album_data or not isinstance(album_data['album'], dict):
            return "Album not found or no data available."

        # Extracting album genres
        album_tags = album_data['album'].get('tags', {}).get('tag', [])
        album_genres = [tag['name'] for tag in album_tags if 'name' in tag]

        tracks = album_data['album'].get('tracks', {}).get('track', [])
        if not tracks:
            return {"Album Genres": album_genres, "Tracks": "No tracks found for the album."}

        # Initialize the result dictionary with album genres
        result = {"Album Genres": album_genres}

        # Adding genres for each track
        for track in tracks:
            track_name = track.get('name', 'Unknown Track')
            track_info_params = {
                "method": "track.getInfo",
                "api_key": api_key,
                "artist": artist,
                "track": track_name,
                "format": "json"
            }

            track_response = requests.get(url, params=track_info_params)
            track_response.raise_for_status()
            track_data = track_response.json()

            if 'track' in track_data and 'toptags' in track_data['track']:
                tags = track_data['track']['toptags'].get('tag', [])
                genres = [tag['name'] for tag in tags if 'name' in tag]
                result[track_name] = genres
            else:
                result[track_name] = ["No genre tags available"]

        return result

    except requests.RequestException as e:
        return f"An error occurred: {e}"


In [59]:
cre

In [110]:
def add_audio_features(df, folder_path):
    for index, row in df.iterrows():
        file_path = row['file_path']  # Use the relative path
        if not os.path.exists(file_path):
            print(f"File not found: {file_path}")
            continue
        features = extract_features(file_path)
        for key, value in features.items():
            if isinstance(value, np.ndarray):  # If the feature is an array
                # Example: Convert array to a comma-separated string
                value = ','.join(map(str, value))
            df.at[index, key] = value


In [156]:
import Utils as u

In [157]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [136]:
creds = u.load_credentials("discogs_api_creds.json")

In [159]:
df_uttu.head()

,title,artist,album,year,genre,label,file_path,tempo,timbre
0,Modul8 (ItaloJohnson Remix),DJ Haus,None,2019,None,Unknown to the Unknown,/mnt/d/UTTU/DJ Haus - Modul8 (ItaloJohnson Rem...,129.199219,"-199.38026,88.61918,34.8789,26.653543,11.23067..."
1,Shadow Man,Neil Landstrumm,None,2019,None,Unknown to the Unknown,/mnt/d/UTTU/Neil Landstrumm - Shadow Man.mp3,129.199219,"-114.02061,81.57262,14.780435,17.86424,15.3996..."
2,Fluid Funk,Elisa Bee,Fluid Funk,2023,None,Unknown to the Unknown,/mnt/d/UTTU/'/Elisa Bee - Fluid Funk - 01 Flui...,112.347147,"-129.48328,63.040688,-4.4237905,43.902008,22.4..."
3,Yes Yes,Elisa Bee,Fluid Funk,2023,None,Unknown to the Unknown,/mnt/d/UTTU/'/Elisa Bee - Fluid Funk - 02 Yes ...,112.347147,"-58.096172,53.636196,-8.037225,16.690744,6.209..."
4,Pins and Needles,Elisa Bee,Fluid Funk,2023,None,Unknown to the Unknown,/mnt/d/UTTU/'/Elisa Bee - Fluid Funk - 03 Pins...,143.554688,"-77.44738,51.692883,2.159064,27.919512,6.98238..."


In [150]:
for index, row in subset_df.iterrows():
    # Use credentials and details to search for a release
    search_results = u.search_release(
        creds['api_key'],
        creds['api_secret'],
        track_title=row['title'],
        artist_name=row['artist'],
        release_title=row['album'],
        year=row.get('year')
    )

    if search_results is None:
        continue
    # Get the release ID from the search results
    release_id = u.get_release_id(search_results)
    if release_id:
        # Fetch genres for this release ID
        genres, styles = u.get_release_details(creds['api_key'], creds['api_secret'], release_id)
        # Update the DataFrame with the genres
        subset_df.at[index, 'genres (discogs)'] = ', '.join(genres)
        subset_df.at[index, 'styles (discogs)'] = ', '.join(styles)
    else:
        subset_df.at[index, 'genres (discogs)'] = 'Not Found'

No results found for DJ Haus - None - Modul8 (ItaloJohnson Remix)


/tmp/ipykernel_5365/4052625562.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df.at[index, 'genres (discogs)'] = ', '.join(genres)
/tmp/ipykernel_5365/4052625562.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df.at[index, 'styles (discogs)'] = ', '.join(styles)


In [164]:
processed_uttu_df = u.process_dataframe(df_uttu, creds)

No results found for 3STRANGE, Alan Fitzpatrick - Cyclone Jazz EP - None
No results found for 3STRANGE, Alan Fitzpatrick - Cyclone Jazz EP - None
No results found for 3STRANGE, Alan Fitzpatrick - Cyclone Jazz EP - None
No results found for 3STRANGE, Alan Fitzpatrick - Cyclone Jazz EP - None
No results found for 3STRANGE, Alan Fitzpatrick - Cyclone Jazz EP - None
No results found for 5kinandBone5 - Make U Understand / Reset - None
No results found for 5kinandBone5 - Make U Understand / Reset - None
No results found for 9th House - 9th House - Phoenica + Loods Remix - None
No results found for 9th House - 9th House - Phoenica + Loods Remix - None
No results found for 9th House - 9th House - Phoenica + Loods Remix - None
No results found for 9th House - Planetary EP (Feat. Baltra & Fear-E) - None
No results found for 9th House - Planetary EP (Feat. Baltra & Fear-E) - None
No results found for 9th House - Planetary EP (Feat. Baltra & Fear-E) - None
No results found for 9th House - Planetar

In [165]:
processed_uttu_df.to_csv("processed_uttu.csv")

In [166]:
processed_hessle_df = u.process_dataframe(df_hessle, creds)
processed_hessle_df.to_csv("processed_hessle.csv")

No results found for Elgato - 116 And Rising (Hessle Audio) - None
No results found for Untold - 116 And Rising (Hessle Audio) - None
No results found for Blawan - 116 And Rising (Hessle Audio) - None
No results found for Pearson Sound - 116 And Rising (Hessle Audio) - None
No results found for Joe - 116 And Rising (Hessle Audio) - None
No results found for Randomer - 116 And Rising (Hessle Audio) - None
No results found for Pangaea - 116 And Rising (Hessle Audio) - None
No results found for Cosmin TRG - 116 And Rising (Hessle Audio) - None
No results found for D1 - 116 And Rising (Hessle Audio) - None
No results found for Addison Groove - 116 And Rising (Hessle Audio) - None
No results found for James Blake - 116 And Rising (Hessle Audio) - None
No results found for Peverelist - 116 And Rising (Hessle Audio) - None
No results found for Pangaea - 116 And Rising (Hessle Audio) - None
No results found for Untold - 116 And Rising (Hessle Audio) - None
No results found for Blawan - 116 And